https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [3]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [2]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_032022_032023_taxi_rides.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

# Create the lagged features 24 +1
lags = 25

# Create the lagged features
for i in range(1, lags):
    data['lag' + str(i)] = data['count'].shift(i)

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [6]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

print(cutoffs)

# Define cutoffs for cross validation with 5 folds every two months with first cut on 2022-06-01, last day 2023-01-15 and horizon of 15 days
#cutoffs = pd.to_datetime(['2022-06-01', '2022-08-01', '2022-10-01', '2022-12-01', '2023-01-15'])
# print(cutoffs)


0       2022-04-02 00:00:00
1       2022-04-02 01:00:00
2       2022-04-02 02:00:00
3       2022-04-02 03:00:00
4       2022-04-02 04:00:00
               ...         
8731    2023-03-31 19:00:00
8732    2023-03-31 20:00:00
8733    2023-03-31 21:00:00
8734    2023-03-31 22:00:00
8735    2023-03-31 23:00:00
Name: ds, Length: 8736, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 1453 1454 1455]
  Test:  index=[1456 1457 1458 ... 2909 2910 2911]
Fold 1:
  Train: index=[   0    1    2 ... 2909 2910 2911]
  Test:  index=[2912 2913 2914 ... 4365 4366 4367]
Fold 2:
  Train: index=[   0    1    2 ... 4365 4366 4367]
  Test:  index=[4368 4369 4370 ... 5821 5822 5823]
Fold 3:
  Train: index=[   0    1    2 ... 5821 5822 5823]
  Test:  index=[5824 5825 5826 ... 7277 7278 7279]
Fold 4:
  Train: index=[   0    1    2 ... 7277 7278 7279]
  Test:  index=[7280 7281 7282 ... 8733 8734 8735]
['2022-06-01 15:00:00', '2022-08-01 07:00:00', '2022-09-30 23:00:00', '2022-11-30 15:00:00', '2023-01-30

In [7]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    train = pd.DataFrame(train_data)
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
     # add regressors with lags 1 to 24
    for i in range(1,25):
        train_model.add_regressor('lag'+str(i), standardize=False)
        train['lag'+str(i)] = train['y'].shift(i)


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")

     # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    


    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


11:55:59 - cmdstanpy - INFO - Chain [1] start processing
11:55:59 - cmdstanpy - INFO - Chain [1] done processing
11:56:01 - cmdstanpy - INFO - Chain [1] start processing
11:56:01 - cmdstanpy - INFO - Chain [1] done processing
11:56:01 - cmdstanpy - INFO - Chain [1] start processing
11:56:01 - cmdstanpy - INFO - Chain [1] start processing
11:56:01 - cmdstanpy - INFO - Chain [1] done processing
11:56:01 - cmdstanpy - INFO - Chain [1] start processing
11:56:01 - cmdstanpy - INFO - Chain [1] start processing
11:56:01 - cmdstanpy - INFO - Chain [1] done processing
11:56:02 - cmdstanpy - INFO - Chain [1] done processing
11:56:02 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 2.51}


11:56:03 - cmdstanpy - INFO - Chain [1] start processing
11:56:03 - cmdstanpy - INFO - Chain [1] done processing
11:56:05 - cmdstanpy - INFO - Chain [1] start processing
11:56:05 - cmdstanpy - INFO - Chain [1] start processing
11:56:05 - cmdstanpy - INFO - Chain [1] done processing
11:56:05 - cmdstanpy - INFO - Chain [1] start processing
11:56:05 - cmdstanpy - INFO - Chain [1] start processing
11:56:05 - cmdstanpy - INFO - Chain [1] done processing
11:56:05 - cmdstanpy - INFO - Chain [1] start processing
11:56:06 - cmdstanpy - INFO - Chain [1] done processing
11:56:06 - cmdstanpy - INFO - Chain [1] done processing
11:56:06 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 5.01}


...

13:27:59 - cmdstanpy - INFO - Chain [1] start processing
13:28:00 - cmdstanpy - INFO - Chain [1] done processing
13:28:01 - cmdstanpy - INFO - Chain [1] start processing
13:28:01 - cmdstanpy - INFO - Chain [1] start processing
13:28:01 - cmdstanpy - INFO - Chain [1] start processing
13:28:01 - cmdstanpy - INFO - Chain [1] start processing
13:28:01 - cmdstanpy - INFO - Chain [1] start processing
13:28:01 - cmdstanpy - INFO - Chain [1] done processing
13:28:02 - cmdstanpy - INFO - Chain [1] done processing
13:28:03 - cmdstanpy - INFO - Chain [1] done processing
13:28:03 - cmdstanpy - INFO - Chain [1] done processing
13:28:04 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 7.51}


13:28:05 - cmdstanpy - INFO - Chain [1] start processing
13:28:06 - cmdstanpy - INFO - Chain [1] done processing
13:28:07 - cmdstanpy - INFO - Chain [1] start processing
13:28:07 - cmdstanpy - INFO - Chain [1] start processing
13:28:07 - cmdstanpy - INFO - Chain [1] start processing
13:28:07 - cmdstanpy - INFO - Chain [1] start processing
13:28:07 - cmdstanpy - INFO - Chain [1] start processing
13:28:08 - cmdstanpy - INFO - Chain [1] done processing
13:28:08 - cmdstanpy - INFO - Chain [1] done processing
13:28:09 - cmdstanpy - INFO - Chain [1] done processing
13:28:09 - cmdstanpy - INFO - Chain [1] done processing


     changepoint_prior_scale  holidays_prior_scale seasonality_mode  \
0                      0.001                  0.01   multiplicative   
1                      0.001                  0.01   multiplicative   
2                      0.001                  0.01   multiplicative   
3                      0.001                  0.01   multiplicative   
4                      0.001                  0.01         additive   
..                       ...                   ...              ...   
155                    0.401                  7.51   multiplicative   
156                    0.401                  7.51         additive   
157                    0.401                  7.51         additive   
158                    0.401                  7.51         additive   
159                    0.401                  7.51         additive   

     seasonality_prior_scale      mase  
0                       0.01  0.691417  
1                       2.51  0.675643  
2                       

13:28:10 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 7.51, 'seasonality_mode': 'additive', 'holidays_prior_scale': 5.01, 'changepoint_prior_scale': 0.001}
Best MASE 0.6557632711921638
[0.6914172685032114, 0.67564266316579, 0.6714223225535342, 0.6726073832010269, 0.658581358489024, 0.6588521120981858, 0.6571954240510819, 0.6595136047740265, 0.6884526323123822, 0.6876989661994276, 0.6752171145521317, 0.6881034550237619, 0.664133745450748, 0.6586481526656952, 0.6566765351167798, 0.6612089562392371, 0.6883927071014826, 0.6905672440010747, 0.6778874846498911, 0.6884661108425947, 0.6617175344874092, 0.656702142558672, 0.6593279759308187, 0.6557632711921638, 0.6884890049414382, 0.6871321229414536, 0.6872171965939134, 0.688898253798594, 0.6630063650417158, 0.6605816972740084, 0.6614545133932989, 0.6638313210308231, 0.714262052785779, 0.6777382152130499, 0.6777002384942159, 0.6781087640529014, 0.6569395570893807, 0.6604925798417097, 0.6607180378138718, 0.6610631897886385, 0.7052927763221717, 0.673683180